IAMSI -- 2021-2022
--------


# TME 09 : Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**

DAM Damien, DURAND Enzo

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-09.tgz** crée le répertoire <code>tme09</code> contenant :
- ce notebook que vous lisez en ce moment et que vous allez compléter.
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
    - <code>apriori</code> : https://borgelt.net/apriori.html
    - <code>fpgrowth</code> : https://borgelt.net/fpgrowth.html



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [1]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5 ou une version supérieure

['/Users/trung/Desktop/M1/IAMSI/tme-09',
 '/Users/trung/opt/anaconda3/lib/python37.zip',
 '/Users/trung/opt/anaconda3/lib/python3.7',
 '/Users/trung/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/trung/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/trung/opt/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/trung/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/trung/.ipython',
 '/Users/trung/.local/lib/python3.7/site-packages']

## Traitement d'une base d'apprentissage


### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [2]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [43]:
def chargeBase(fname:str,delimiteur=' '):
	d = dict()
	with open(fname, 'r') as f:
		lecteur = csv.reader(f, delimiter=delimiteur)
		i = 0
		for ligne in lecteur:
			d[i] = set(ligne)
			i += 1
	return d

chargeBase('datasets/exemple-1.txt')

{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

In [5]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [6]:
def noms_items(base):
	ens = set()
	for t in base:
		ens = ens.union(base[t])
	return ens

In [7]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [8]:
def singletons(base):
	items = noms_items(base)
	return [{item} for item in items]

In [9]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'d'}, {'b'}, {'a'}, {'c'}, {'e'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [35]:
def comptage(base, itemset):
	n = 0
	for t in base:
		n += (itemset <= base[t])
	return n

In [13]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [14]:
def support(base, itemset):
	return comptage(base, itemset) / len(base)

In [15]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [32]:
from typing import List, Set
from itertools import combinations

def apriori_gen(list_itemsets: List[Set[str]]) -> List[Set[str]]:
	res = set()
	k = len(list_itemsets[0])
	for set1, set2 in combinations(list_itemsets, 2):
		c = set1.union(set2)
		if len(c) != k + 1:
			continue
		for i in c:
			candidat = c - {i}
			if candidat not in list_itemsets:
				break
		else:
			res.add(frozenset(c))
	return [set(c) for c in res]

In [34]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'c', 'd'}, {'c', 'b'}, {'c', 'a'}, {'d', 'b'}, {'d', 'a'}, {'b', 'a'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'d', 'b', 'a'}, {'c', 'd', 'b'}]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [38]:
def apriori(base, minsup:float):
	Ck = singletons(base)
	F = []
	while len(Ck):
		Fk = []
		for c in Ck:
			supc = comptage(base, c) / len(base)
			if supc >= minsup:
				Fk.append(c)
				F.append((c, supc))
		Ck = apriori_gen(Fk)
	return F

In [39]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'d'}, 0.7), ({'b'}, 0.7), ({'a'}, 0.6), ({'c'}, 0.7), ({'e'}, 0.3), ({'c', 'd'}, 0.4), ({'c', 'b'}, 0.6), ({'c', 'a'}, 0.4), ({'d', 'b'}, 0.4), ({'d', 'a'}, 0.4), ({'b', 'a'}, 0.5), ({'e', 'd'}, 0.3), ({'c', 'b', 'a'}, 0.4), ({'c', 'd', 'b'}, 0.3), ({'d', 'b', 'a'}, 0.3)]


In [16]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'c'}, 0.7), ({'a'}, 0.6), ({'d'}, 0.7), ({'b'}, 0.7), ({'e'}, 0.3), ({'c', 'a'}, 0.4), ({'d', 'c'}, 0.4), ({'b', 'c'}, 0.6), ({'d', 'a'}, 0.4), ({'b', 'a'}, 0.5), ({'b', 'd'}, 0.4), ({'e', 'd'}, 0.3), ({'b', 'c', 'a'}, 0.4), ({'b', 'd', 'c'}, 0.3), ({'b', 'd', 'a'}, 0.3)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [40]:
BaseTD = chargeBase('datasets/td.txt')

In [41]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'d'}, 0.9), ({'b'}, 0.7), ({'a'}, 0.5), ({'c'}, 0.5), ({'e'}, 0.6), ({'c', 'd'}, 0.4), ({'e', 'b'}, 0.4), ({'c', 'b'}, 0.3), ({'e', 'a'}, 0.4), ({'d', 'b'}, 0.6), ({'d', 'a'}, 0.4), ({'b', 'a'}, 0.3), ({'e', 'd'}, 0.6), ({'e', 'd', 'a'}, 0.4), ({'e', 'd', 'b'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable <code>apriori</code>.

In [44]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiteur=',')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [45]:
print("Pour la BASE précédente :")
print(noms_items(Base_mushrooms))
print("Exemple: singletons(Base_mushrooms) rend :")
print(singletons(Base_mushrooms))

Pour la BASE précédente :
{'KNOBBED', 'NARROW', 'TAPERING', 'SPICY', 'CLUB', 'SMOOTH', 'FIBROUS', 'BROWN', 'GROOVES', 'URBAN', 'BRUISES', 'NO', 'CROWDED', 'ATTACHED', 'CONICAL', 'ORANGE', 'EDIBLE', 'EQUAL', 'RED', 'BUFF', 'BULBOUS', 'BELL', 'CONVEX', 'LEAVES', 'TWO', 'GRASSES', 'PARTIAL', 'BROAD', 'PENDANT', 'CLOSE', 'GRAY', 'SOLITARY', 'ONE', 'FISHY', 'MEADOWS', 'SCATTERED', 'FREE', 'PURPLE', 'CHOCOLATE', 'ANISE', 'WASTE', 'SILKY', 'BLACK', 'ENLARGING', 'CLUSTERED', 'MUSTY', 'SCALY', 'SUNKEN', 'PINK', 'ROOTED', '?', 'CREOSOTE', 'FLARING', 'ALMOND', 'LARGE', 'EVANESCENT', 'SEVERAL', 'NONE', 'FLAT', 'GREEN', 'PATHS', 'ABUNDANT', 'WOODS', 'PUNGENT', 'FOUL', 'WHITE', 'NUMEROUS', 'POISONOUS', 'CINNAMON', 'YELLOW'}
Exemple: singletons(Base_mushrooms) rend :
[{'KNOBBED'}, {'NARROW'}, {'TAPERING'}, {'SPICY'}, {'CLUB'}, {'SMOOTH'}, {'FIBROUS'}, {'BROWN'}, {'GROOVES'}, {'URBAN'}, {'BRUISES'}, {'NO'}, {'CROWDED'}, {'ATTACHED'}, {'CONICAL'}, {'ORANGE'}, {'EDIBLE'}, {'EQUAL'}, {'RED'}, {'BUFF'}, {

In [46]:
print(apriori(Base_mushrooms,0.9))

[({'PARTIAL'}, 1.0), ({'ONE'}, 0.9230038022813688), ({'FREE'}, 0.9743346007604563), ({'WHITE'}, 0.9771863117870723), ({'ONE', 'WHITE'}, 0.9001901140684411), ({'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'WHITE'}, 0.9771863117870723), ({'FREE', 'WHITE'}, 0.9743346007604563), ({'PARTIAL', 'ONE', 'WHITE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'ONE'}, 0.9001901140684411), ({'FREE', 'ONE', 'WHITE'}, 0.9001901140684411), ({'PARTIAL', 'WHITE', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'FREE', 'ONE', 'WHITE'}, 0.9001901140684411)]


In [21]:
print(apriori(Base_mushrooms,0.9))

[({'WHITE'}, 0.9771863117870723), ({'PARTIAL'}, 1.0), ({'FREE'}, 0.9743346007604563), ({'ONE'}, 0.9230038022813688), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'FREE', 'ONE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411), ({'WHITE', 'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'ONE'}, 0.9001901140684411), ({'FREE', 'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411)]


In [47]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.csv',delimiteur=',')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [48]:
# décommenter les lignes suivantes:

print("Pour la BASE Titanic :")
print(noms_items(Base_titanic))

print("Exemple: singletons(Base_titanic) rend :")
print(singletons(Base_titanic))

Pour la BASE Titanic :
{'StockholmSwedenNewYork', 'GunnislakeEnglandButteMT', 'HalesworthEngland', 'VancouverBC', 'StIvesCornwallHoughtonMI', 'IsleofWightEngland', 'TreherbertCardiffWales', 'LondonEngland', 'OsloNorwayCameronWI', 'LondonWinnipegMB', 'KrakorydSwedenBloomingtonIL', 'StLeonardsonSeaEnglandOhio', 'GlenRidgeNJ', 'NewYorkNYBriarcliffManorNY', 'EnglandSacramentoCA', 'EastProvidenceRI', 'GuernseyEnglandEdgewoodRI', 'WestKensingtonLondon', 'EnglandHartfordCT', 'PlymouthDorsetHoughtonMI', 'JacksonvilleFL', 'WestonSuperMareMooseJawSK', 'GreensburgPA', 'BarreCoWashingtonVT', 'AustraliaFingalND', 'WestHavenCT', 'TorontoON', 'KrakuddenSwedenMouneIL', 'ItalyPhiladelphiaPA', 'FondduLacWI', 'HolleyNY', 'HartfordHuntingdonshire', 'KilmacowenCoSligoIrelandNewYorkNY', 'EnglandBrooklynNY', 'RotterdamNetherlands', 'Cornwall', 'pclass', 'CoClareIrelandWashingtonDC', 'AlbanyNY', 'BronxNY', 'BrunswickME', 'GoteborgSwedenRockfordIL', 'ForesvikNorwayPortlandND', 'EastOrangeNJ', 'CooperstownNY', 

In [49]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
#print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
#print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


({'male', 'dead'}, 0.46963562753036436)
({'male', 'Southampton'}, 0.46288798920377866)
({'dead', 'Southampton'}, 0.41295546558704455)
({'male', 'dead', 'Southampton'}, 0.3684210526315789)
({'female', 'alive'}, 0.32388663967611336)
({'2nd', 'Southampton'}, 0.30634278002699056)
({'alive', 'Southampton'}, 0.2982456140350877)
({'female', 'Southampton'}, 0.2483130904183536)
({'1st', 'alive'}, 0.23076923076923078)
({'male', '1st'}, 0.21997300944669365)
({'1st', 'Southampton'}, 0.20917678812415655)
({'male', '2nd'}, 0.20647773279352227)
({'female', 'alive', 'Southampton'}, 0.203778677462888)


In [51]:
Lres = apriori(Base_titanic,0.2)

print("tri sur les valeurs de supports:")
for t in sorted(Lres, key=lambda data: data[1]):
	if len(t[0])>1:
		print(t)

print("\ntri sur la taille des itemsets")
for t in sorted(Lres, key=lambda data: len(data[0])):
	if len(t[0])>1:
		print(t)

print()
for t in sorted(Lres, key=lambda data: data[1],reverse = True):
	if len(t[0])>1:
		print(t)


tri sur les valeurs de supports:
({'female', 'alive', 'Southampton'}, 0.203778677462888)
({'male', '2nd'}, 0.20647773279352227)
({'1st', 'Southampton'}, 0.20917678812415655)
({'male', '1st'}, 0.21997300944669365)
({'1st', 'alive'}, 0.23076923076923078)
({'female', 'Southampton'}, 0.2483130904183536)
({'alive', 'Southampton'}, 0.2982456140350877)
({'2nd', 'Southampton'}, 0.30634278002699056)
({'female', 'alive'}, 0.32388663967611336)
({'male', 'dead', 'Southampton'}, 0.3684210526315789)
({'dead', 'Southampton'}, 0.41295546558704455)
({'male', 'Southampton'}, 0.46288798920377866)
({'male', 'dead'}, 0.46963562753036436)

tri sur la taille des itemsets
({'1st', 'Southampton'}, 0.20917678812415655)
({'2nd', 'Southampton'}, 0.30634278002699056)
({'female', 'alive'}, 0.32388663967611336)
({'1st', 'alive'}, 0.23076923076923078)
({'male', '1st'}, 0.21997300944669365)
({'female', 'Southampton'}, 0.2483130904183536)
({'male', '2nd'}, 0.20647773279352227)
({'male', 'Southampton'}, 0.46288798920377

<font color="RED" size="+1">**[Q]**</font>  Utiliser l'éxécutable <code>apriori</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [52]:
!./apriori datasets/mushrooms.txt -s90 res.txt
!cat res.txt

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [4/8416 transaction(s)] done [0.00s].
building transaction tree ... [6 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing res.txt ... [15 set(s)] done [0.00s].
PARTIAL (100)
ONE PARTIAL (92.3004)
ONE (92.3004)
ONE FREE PARTIAL WHITE (90.019)
ONE FREE PARTIAL (90.019)
ONE FREE WHITE (90.019)
ONE FREE (90.019)
ONE WHITE PARTIAL (90.019)
ONE WHITE (90.019)
FREE PARTIAL WHITE (97.4335)
FREE PARTIAL (97.4335)
FREE WHITE (97.4335)
FREE (97.4335)
WHITE PARTIAL (97.7186)
WHITE (97.7186)


In [54]:
!./apriori datasets/titanic-red.csv -s20 res.txt
!cat res.txt

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [9 item(s)] done [0.00s].
sorting and reducing transactions ... [32/741 transaction(s)] done [0.00s].
building transaction tree ... [48 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing res.txt ... [22 set(s)] done [0.00s].
Cherbourg (23.7517)
3rd (27.1255)
2nd (34.8178)
2nd male (20.6478)
2nd Southampton (30.6343)
1st (37.9217)
1st alive (23.0769)
1st male (21.9973)
1st Southampton (20.9177)
female (38.8664)
female alive (32.3887)
female alive Southampton (20.3779)
female Southampton (24.8313)
alive (46.4238)
alive Southampton (29.8246)
dead (53.4413)
dead male (46.9636)
dead male Southampton (36.8421)
dead Southampton (41.2955)
male (60.9987)
male Southampton (46.2888)
Southampton (71.1201)


<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


In [57]:
!./apriori datasets/titanic-red.csv -s20 -el res.txt
!cat res.txt

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [9 item(s)] done [0.00s].
sorting and reducing transactions ... [32/741 transaction(s)] done [0.00s].
building transaction tree ... [48 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing res.txt ... [13 set(s)] done [0.00s].
2nd male (20.6478)
2nd Southampton (30.6343)
1st alive (23.0769)
1st male (21.9973)
1st Southampton (20.9177)
female alive (32.3887)
female alive Southampton (20.3779)
female Southampton (24.8313)
alive Southampton (29.8246)
dead male (46.9636)
dead male Southampton (36.8421)
dead Southampton (41.2955)
male Southampton (46.2888)


In [61]:
!./apriori datasets/mushrooms.txt -s90 -el res.txt
!cat res.txt

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [4/8416 transaction(s)] done [0.00s].
building transaction tree ... [6 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing res.txt ... [11 set(s)] done [0.00s].
ONE FREE (90.019)
ONE FREE WHITE (90.019)
ONE FREE WHITE PARTIAL (90.019)
ONE FREE PARTIAL (90.019)
ONE WHITE (90.019)
ONE WHITE PARTIAL (90.019)
ONE PARTIAL (92.3004)
FREE WHITE (97.4335)
FREE WHITE PARTIAL (97.4335)
FREE PARTIAL (97.4335)
WHITE PARTIAL (97.7186)


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \text {Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y)  \\
 \text {IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}}
\end{align}

### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [66]:
!time ./apriori datasets/titanic-red.csv -s20 -el
!time ./fpgrowth datasets/titanic-red.csv -s20 -el

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [9 item(s)] done [0.00s].
sorting and reducing transactions ... [32/741 transaction(s)] done [0.00s].
building transaction tree ... [48 node(s)] done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing <null> ... [13 set(s)] done [0.00s].

real	0m0.008s
user	0m0.002s
sys	0m0.003s
./fpgrowth - find frequent item sets with the fpgrowth algorithm
version 6.20 (2022.01.09)        (c) 2004-2022   Christian Borgelt
reading datasets/titanic-red.csv ... [378 item(s), 741 transaction(s)] done [0.00s].
filtering, sorting and recoding items ... [9 item(s)] done [0.00s].
sorting and reducing transactions ... [32/741 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing <null> ... [13 set(s)] done [0.00s].

In [67]:
!time ./apriori datasets/mushrooms.txt -s90 -el
!time ./fpgrowth datasets/mushrooms.txt -s90 -el

./apriori - find frequent item sets with the apriori algorithm
version 6.30 (2020.01.09)        (c) 1996-2022   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [4/8416 transaction(s)] done [0.00s].
building transaction tree ... [6 node(s)] done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing <null> ... [11 set(s)] done [0.00s].

real	0m0.021s
user	0m0.014s
sys	0m0.004s
./fpgrowth - find frequent item sets with the fpgrowth algorithm
version 6.20 (2022.01.09)        (c) 2004-2022   Christian Borgelt
reading datasets/mushrooms.txt ... [70 item(s), 8416 transaction(s)] done [0.01s].
filtering, sorting and recoding items ... [4 item(s)] done [0.00s].
sorting and reducing transactions ... [4/8416 transaction(s)] done [0.00s].
finding frequent item set(s) ... done [0.00s].
writing <null> ... [11 set(s)] done [0.00s].

r

> On peut constater que fpgrowth est legerement plus efficace qu'apriori

<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des exécutables *apriori* et *fpgrowth*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori*, soit *fpgrowth*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
